<a href="https://colab.research.google.com/github/jeffvestal/elastic_jupyter_notebooks/blob/main/load_embedding_model_from_hf_to_elastic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading an NLP Embedding model from Hugging Face into Elastic

This code will show you how to load a supported embedding model from Hugging Face into an elasticsearch cluster in [Elastic Cloud](https://cloud.elastic.co/)

### Elastic version support
Requires Elastic version 8.0+ with a platinum or enterprise license (or trial license)

You can set up a [free trial elasticsearch Deployment in Elastic Cloud](https://cloud.elastic.co/registration).

## Deleteme below

Example here is loading a [Zero Shot model](https://huggingface.co/typeform/distilbert-base-uncased-mnli)

[Elastic NLP Model Support Docs](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-model-ref.html) 


Disclaimer: presented as is with no guarantee.

# Install and import required python libraries

Elastic uses the [eland python library](https://github.com/elastic/eland) to download modesl from Hugging Face hub and load them into elasticsearch

In [1]:
pip install eland

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.4/385.4 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 10.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.3 MB/s eta 0:00:00


In [6]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c8ee7abe71bd203971a09c508a080912c39f61b2f2e80e47ff67f2d0de465229
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [ ]:
pip install torch==1.11

In [7]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

In [21]:

from elasticsearch.client import MlClient


<ipython-input-21-8ee2a6f3019e>:1: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


# Configure elasticsearch authentication. 
For this example we are using the [Elastic Cloud ID](https://www.elastic.co/guide/en/cloud/current/ec-cloud-id.html) and a [cluster API key](https://www.elastic.co/guide/en/kibana/current/api-keys.html)

You can use any method you wish to set the required credentials. We are using getpass in this example to prompt for credentials.

In [8]:
import getpass

In [15]:
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
es_api_id = getpass.getpass('Enter cluster API key ID:  ') 
es_api_key = getpass.getpass('Enter cluster API key:  ')

Enter Elastic Cloud ID:  ··········
Enter cluster API key ID:  ··········
Enter cluster API key:  ··········


# Connect to Elastic and Load a Hugging Face Model

In [16]:
es = Elasticsearch(cloud_id=es_cloud_id, 
                   api_key=(es_api_id, es_api_key)
                   )
es.info() # should return cluster info

ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': 'a7bf48bf42ad403ab45dd6b90b860f85', 'cluster_uuid': 'gEbjuhUOSyCVzG4Gz2SQ2w', 'version': {'number': '8.6.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'f67ef2df40237445caa70e2fef79471cc608d70d', 'build_date': '2023-01-04T09:35:21.782467981Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

[Supported `task_type` values](https://github.com/elastic/eland/blob/15a300728876022b206161d71055c67b500a0192/eland/ml/pytorch/transformers.py#*L41*)

# Download an embedding model from Hugging Face using the HF copy link

[sentence-transformers/msmarco-MiniLM-L-12-v3](https://huggingface.co/sentence-transformers/msmarco-MiniLM-L-12-v3)


In [33]:
hf_model_id='sentence-transformers/msmarco-MiniLM-L-12-v3'
tm = TransformerModel(hf_model_id, "text_embedding")

In [36]:
es_model_id = tm.elasticsearch_model_id()
es_model_id

'sentence-transformers__msmarco-minilm-l-12-v3'

### Export the model in a TorchScrpt representation which Elasticsearch uses

In [35]:
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

### Import model into Elasticsearch
Model should not already exist in elasticsearch

In [37]:
ptm = PyTorchModel(es, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config) 

  0%|          | 0/32 [00:00<?, ? parts/s]

# Operationalizing the Model

## Deploy the model
This will load the model on the ML nodes for use

In [49]:
# List the in elasticsearch
m = MlClient.get_trained_models(es, model_id=es_model_id)
m.body

{'count': 1,
 'trained_model_configs': [{'model_id': 'sentence-transformers__msmarco-minilm-l-12-v3',
   'model_type': 'pytorch',
   'created_by': 'api_user',
   'version': '8.6.0',
   'create_time': 1675720672194,
   'model_size_bytes': 0,
   'estimated_operations': 0,
   'license_level': 'platinum',
   'description': "Model sentence-transformers/msmarco-MiniLM-L-12-v3 for task type 'text_embedding'",
   'tags': [],
   'input': {'field_names': ['text_field']},
   'inference_config': {'text_embedding': {'vocabulary': {'index': '.ml-inference-native-000001'},
     'tokenization': {'bert': {'do_lower_case': True,
       'with_special_tokens': True,
       'max_sequence_length': 512,
       'truncate': 'first',
       'span': -1}}}},
   'location': {'index': {'name': '.ml-inference-native-000001'}}}]}

In [48]:
# start trained model deployment
s = MlClient.start_trained_model_deployment(es, model_id=es_model_id)
s.body

# You can see model state in Kibana -> Machine Learning -> Model Management -> Trained Models

{'assignment': {'task_parameters': {'model_id': 'sentence-transformers__msmarco-minilm-l-12-v3',
   'model_bytes': 132922074,
   'threads_per_allocation': 1,
   'number_of_allocations': 1,
   'queue_capacity': 1024,
   'cache_size': '132922074b',
   'priority': 'normal'},
  'routing_table': {'dDqmmPySSKuH8d7HcF54uA': {'current_allocations': 1,
    'target_allocations': 1,
    'routing_state': 'started',
    'reason': ''}},
  'assignment_state': 'started',
  'start_time': '2023-02-06T23:39:10.493431663Z',
  'max_assigned_allocations': 1}}

In [65]:
stats = MlClient.get_trained_models_stats(es, model_id=es_model_id)
stats.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']

{'routing_state': 'failed',
 'reason': 'inference process crashed due to reason [[sentence-transformers__msmarco-minilm-l-12-v3] pytorch_inference/39784 process stopped unexpectedly: Error loading the model: version_ <= kMaxSupportedFileFormatVersionINTERNAL ASSERT FAILED at "/usr/src/pytorch/caffe2/serialize/inline_container.cc":140, please report a bug to PyTorch. Attempted to read a PyTorch file with version 10, but the maximum supported version for reading is 9. Your PyTorch installation may be too old.\n]'}

## Generate Vector for Query


In [43]:
#{
#  "docs": [{"text_field": "What was Jean Valjean prisoner number?"}]
#}

docs =  [
    {
      "text_field": "Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."
    }
  ]

In [46]:
# future reference do not use yet
#z = MlClient.infer_trained_model_deployment(es, model_id =es_model_id, docs=docs, )
z = MlClient.infer_trained_model(es, model_id=es_model_id, docs=docs, )

ConflictError: ignored

In [ ]:
# Using requests until MlClient.infer_trained_model_deployment is updated to accept inference extra configs
import requests
from requests.auth import HTTPBasicAuth
import urllib.parse

endpoint = '_ml/trained_models/%s/deployment/_infer' % model_id
url = urllib.parse.urljoin(es_url, endpoint)

body = {
  "docs": [
    {
      "text_field": "Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."
    }
  ],
  "inference_config": {
    "zero_shot_classification": {
      "labels": [
        "mobile",
        "website",
        "billing",
        "account access"
      ],
      "multi_label": True
    }
  }
}

resp = requests.post(url, auth=HTTPBasicAuth(es_user, es_pass), json=body)
r = resp.json()
print('Predicted value is: %s with a probability of %0.2f%%' % (r['predicted_value'], r['prediction_probability'] * 100))
print('=-=-=-=')
print('Full Probability output:')
for c in r['top_classes']:
    print ('%s probability of %0.5f%%' % (c['class_name'], c['class_probability'] * 100))

In [ ]:
# Just to see the full doc
resp.json()